In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from PIL import Image
import os
import datetime
import csv
from slackclient import SlackClient
from env import *

def slack_message(message, channel):
    token = SLACK_TOKEN
    sc = SlackClient(token)
    sc.api_call('chat.postMessage', channel=channel, 
                text=message, username='queue_update')

def val2key(d, search_v):
    for k,v in d.iteritems():
        if v == search_v:
            return k

driver = webdriver.Chrome('/Users/jskycak/automation/chromedriver_mac')
driver.get('https://cms8.remilon.com/login')

username = driver.find_element_by_name("username")
password = driver.find_element_by_name("password")

username.send_keys("jskycak")
password.send_keys(CMS_PASS)

driver.find_element_by_class_name("btn").click()

previous_num_items_available = 'XXX'
still_running = True
while still_running:
    try:
        driver.get('https://cms8.remilon.com/work#!/available')
        time.sleep(120)
        screenshot = driver.save_screenshot('temp/available_items_screenshot.png')
        screenshot_time = datetime.datetime.now()
        print '------------------------\n[BEGIN]',screenshot_time

        img = Image.open('temp/available_items_screenshot.png')
        width, height = img.size[0], img.size[1]
        area_full = (int(0.761*width), int(0.27*height), int(0.78*width), int(0.32*height))
        area_hundreds = (int(0.761*width), int(0.27*height), int(0.768*width), int(0.32*height))
        area_tens = (int(0.767*width), int(0.27*height), int(0.774*width), int(0.32*height))
        area_ones = (int(0.773*width), int(0.27*height), int(0.78*width), int(0.32*height))

        img.crop(area_full).save('temp/available_items_template_full.png')
        img.crop(area_ones).save('temp/available_items_template_ones.png')
        img.crop(area_tens).save('temp/available_items_template_tens.png')
        img.crop(area_hundreds).save('temp/available_items_template_hundreds.png')

        existing_templates_full_filenames = ['templates/full/'+filename for filename in os.listdir('templates/full')]
        existing_templates_ones_filenames = ['templates/ones/'+filename for filename in os.listdir('templates/ones')]
        existing_templates_tens_filenames = ['templates/tens/'+filename for filename in os.listdir('templates/tens')]
        existing_templates_hundreds_filenames = ['templates/hundreds/'+filename for filename in os.listdir('templates/hundreds')]
        
        template_full = Image.open('temp/available_items_template_full.png')
        template_ones = Image.open('temp/available_items_template_ones.png')
        template_tens = Image.open('temp/available_items_template_tens.png')
        template_hundreds = Image.open('temp/available_items_template_hundreds.png')

        ones_str = 'X'
        tens_str = 'X'
        hundreds_str = 'X'
        save_filename = screenshot_time.strftime("%Y_%m_%d_%H_%M_%S")+".png"
        
        exists_already_full = False
        exists_already_ones = False
        exists_already_tens = False
        exists_already_hundreds = False
        
        for filename in existing_templates_full_filenames:
            with Image.open(filename) as existing_template_full:
                if template_full == existing_template_full:
                    exists_already_full = True
                    hundreds_str = filename.split('/')[-1][0]
                    tens_str = filename.split('/')[-1][1]
                    ones_str = filename.split('/')[-1][2]
                    print 'found full = '+hundreds_str+tens_str+ones_str
                    
        if not exists_already_full:
            
            template_full.save("templates/full/XXX_"+save_filename)
            print "saved templates/full/XXX_"+save_filename
            
            for filename in existing_templates_ones_filenames:
                with Image.open(filename) as existing_template_ones:
                    if template_ones == existing_template_ones:
                        exists_already_ones = True
                        ones_str = filename.split('/')[-1][0]
                        print 'found ones = '+ones_str
            if not exists_already_ones:
                template_ones.save("templates/ones/X_"+save_filename)
                print "saved templates/ones/X_"+save_filename
                        
            for filename in existing_templates_tens_filenames:
                with Image.open(filename) as existing_template_tens:
                    if template_tens == existing_template_tens:
                        exists_already_tens = True
                        tens_str = filename.split('/')[-1][0]
                        print 'found tens = '+tens_str
            if not exists_already_tens:
                template_tens.save("templates/tens/X_"+save_filename)
                print "saved templates/tens/X_"+save_filename
                        
            for filename in existing_templates_hundreds_filenames:
                with Image.open(filename) as existing_template_hundreds:
                    if template_hundreds == existing_template_hundreds:
                        exists_already_hundreds = True
                        hundreds_str = filename.split('/')[-1][0]
                        print 'found hundreds = '+hundreds_str
            if not exists_already_hundreds:
                template_hundreds.save("templates/hundreds/X_"+save_filename)
                print "saved templates/hundreds/X_"+save_filename

        num_items_available = hundreds_str + tens_str + ones_str
        print 'found num_items_available = '+num_items_available

        with open('queue_log.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([screenshot_time, num_items_available])

        try:
            num_items_delta = str(int(num_items_available) - int(previous_num_items_available))
            if int(num_items_delta) > 0:
                slack_message('{0} items added ({1} available)'.format(*[num_items_delta, num_items_available]),'queue_update')
                print '[END] {0} | {1} | +{2}'.format(*[screenshot_time, num_items_available, num_items_delta])
            else: 
                print '[END] {0} | {1} | {2}'.format(*[screenshot_time, num_items_available, num_items_delta])
        except:
            print '[END] {0} | {1} | n/a'.format(*[screenshot_time, num_items_available])

        previous_num_items_available = num_items_available

    except Exception as e:
        print(e)
        slack_message('ERROR - SCRIPT FAILED','queue_update')
        driver.quit()
        still_running = False

------------------------
[BEGIN] 2018-02-05 18:52:17.266263
found full = 261
found num_items_available = 261
[END] 2018-02-05 18:52:17.266263 | 261 | n/a
------------------------
[BEGIN] 2018-02-05 18:54:18.567606
found full = 260
found num_items_available = 260
[END] 2018-02-05 18:54:18.567606 | 260 | -1
------------------------
[BEGIN] 2018-02-05 18:56:19.714559
found full = 260
found num_items_available = 260
[END] 2018-02-05 18:56:19.714559 | 260 | 0
------------------------
[BEGIN] 2018-02-05 18:58:21.041908
found full = 260
found num_items_available = 260
[END] 2018-02-05 18:58:21.041908 | 260 | 0
------------------------
[BEGIN] 2018-02-05 19:00:22.185738
found full = 260
found num_items_available = 260
[END] 2018-02-05 19:00:22.185738 | 260 | 0
------------------------
[BEGIN] 2018-02-05 19:02:23.944348
found full = 259
found num_items_available = 259
[END] 2018-02-05 19:02:23.944348 | 259 | -1
------------------------
[BEGIN] 2018-02-05 19:04:25.052785
found full = 259
found n

In [ ]:
!rm templates/*/.DS_store